# Programa para deducir la matriz de rigidez de un elemento de pórtico de Timoshenko-Ehrenfest a partir de la solución de la ecuación diferencial

In [1]:
import sympy as sp

# Para imprimir bonito
sp.init_printing()

from IPython.display import Math                 
def imprimir (texto, variable, grande=False):
    if grande:
        return Math(texto + r'\Large{' + rf'{sp.latex(variable)}' + '}')
    else:
        return Math(texto +  rf'{sp.latex(sp.simplify(variable))}')

In [2]:
# Definción de variables
x, xi, L, EI, beta, b, EA = sp.symbols('x xi L EI beta b EA')

# Se definen las constantes de integración
C1, C2, C3, C4, C5, C6 = sp.symbols('C_1:7')

# Se define beta como beta = (12 * EI)/(L**2 * GAast)
GAast = (12 * EI)/(L**2 * beta)

# Se define la carga: hace la ecuación diferencial homogénea
q = 0
b = 0

## Cálculo de la matriz de rigidez $\boldsymbol{K}_{TE}$

Recordemos que $\beta = \frac{12  EI}{L^2  GA^*}$ es el *factor de influencia por cortante*. En la viga de Euler-Bernoulli $\beta = 0$.

### Método 1

In [3]:
# Se plantea la ecuación diferencial
V =  sp.integrate(q, x)           + C1  # Ecuaciones calculadas con constantes.
M =  sp.integrate(V, x)           + C2
t =  (sp.integrate(M, x)          + C3)/EI
w =  sp.integrate(t - V/GAast, x) + C4
A = -sp.integrate(b, x)           + C5  # fuerza axial
u =  sp.integrate(A/EA, x)        + C6

# Se calcula la matrix de rigidez
K_TE = sp.zeros(6)
N_u = sp.zeros(1,6)
N_w = sp.zeros(1,6)
N_t = sp.zeros(1,6)
for i in range(6):
    sol = sp.solve([u.subs(x, 0) - int((i == 0)),
                    w.subs(x, 0) - int((i == 1)),  # Condiciones de frontera
                    t.subs(x, 0) - int((i == 2)),
                    u.subs(x, L) - int((i == 3)),
                    w.subs(x, L) - int((i == 4)),
                    t.subs(x, L) - int((i == 5))],
                    [C1, C2, C3, C4, C5, C6])

    constantes = [(C1, sol[C1]), (C2, sol[C2]), (C3, sol[C3]), (C4, sol[C4]), (C5, sol[C5]), (C6, sol[C6])]
    
    sol_A  = A.subs(constantes)
    sol_V  = V.subs(constantes)
    sol_M  = M.subs(constantes)
    N_u[i] = u.subs(constantes)
    N_w[i] = w.subs(constantes)
    N_t[i] = t.subs(constantes)

    # se evaluan las reacciones horizontales y verticales y los momentos en los apoyos
    K_TE[:, i] = [- sol_A.subs(x, 0),   # X1
                  + sol_V.subs(x, 0),   # Y1
                  - sol_M.subs(x, 0),   # M1
                  + sol_A.subs(x, L),   # X2
                  - sol_V.subs(x, L),   # Y2
                  + sol_M.subs(x, L)]   # M2

# Se convierten las funciones de forma a coordenadas naturales
N_u = sp.simplify(N_u.subs(x, L*(1+xi)/2))
N_w = sp.simplify(N_w.subs(x, L*(1+xi)/2))  # x = L*xi/2 + L/2
N_t = sp.simplify(N_t.subs(x, L*(1+xi)/2))
    
# Se imprime la solución
imprimir(r'K_{TE} = ', K_TE)

<IPython.core.display.Math object>

## Cálculo de la matriz de rigidez de la teoría de Euler-Bernoulli $\boldsymbol{K}_{EB}$
Observe que cuando $GA^* \to \infty$ (o alternativamente $\beta \to 0$), la matriz de rigidez $\boldsymbol{K}_{TE}$ se vuelve la misma matriz de rigidez $\boldsymbol{K}_{EB}$ de la teoría de Euler-Bernoulli:

In [4]:
K_EB = K_TE.limit(beta, 0)

# Se imprime la solución
imprimir(r'\boldsymbol{K}_{EB} = ', K_EB)

<IPython.core.display.Math object>

## Se imprimen las funciones de forma "exactas" del EF de Timoshenko-Ehrenfest

In [17]:
imprimir(r'\boldsymbol{N}_{u}(\xi) = ', N_u)

<IPython.core.display.Math object>

In [18]:
imprimir(r'\boldsymbol{N}_{w}(\xi) = ', N_w)

<IPython.core.display.Math object>

In [19]:
imprimir(r'\boldsymbol{N}_{\theta}(\xi) = ', N_t)

<IPython.core.display.Math object>

### Método 2

In [5]:
V = sp.Function('V')(x)
M = sp.Function('M')(x)
t = sp.Function('theta')(x)
w = sp.Function('w')(x)
    
# Se calcula la matrix de rigidez
K_TE = sp.zeros(4)
for i in range(4):
    sol = sp.dsolve(eq=[  sp.diff(V,x) - 0,
                          sp.diff(M,x) - V,
                          sp.diff(t,x) - M/EI,
                          sp.diff(w,x) - t + V/GAast   ],
                    ics={ w.subs(x, 0): int((i == 0)),
                          t.subs(x, 0): int((i == 1)),
                          w.subs(x, L): int((i == 2)),
                          t.subs(x, L): int((i == 3))  })
    VV = sol[0].rhs
    MM = sol[1].rhs
    
    # se evaluan las reacciones horizontales y verticales y los momentos en los apoyos
    K_TE[:, i] = [+ VV.subs(x, 0),   # Y1 
                  - MM.subs(x, 0),   # M1
                  - VV.subs(x, L),   # Y2
                  + MM.subs(x, L)]   # M2

# Se imprime la solución
tmp = EI/((1 + beta)*L**3)
imprimir(r'\boldsymbol{K}_{TE} = \frac{E I}{(1 + \beta) L^3}', K_TE/tmp)

<IPython.core.display.Math object>

## Cálculo del vector de fuerzas nodales equivalentes asociados a una carga trapezoidal

In [6]:
w1, w2 = sp.symbols('w_1 w_2')
q = (w2 - w1)*x/L + w1

# Se plantea la ecuación diferencial
V = sp.integrate(q, x)           + C1  # Ecuaciones calculadas con constantes.
M = sp.integrate(V, x)           + C2
t = (sp.integrate(M, x)          + C3)/EI
w = sp.integrate(t - V/GAast, x) + C4

# Se calcula la matrix de rigidez
sol = sp.solve([w.subs(x, 0),  # Condiciones de frontera
                w.subs(x, L),
                t.subs(x, 0),
                t.subs(x, L)],
                [C1, C2, C3, C4])

constantes = [(C1, sol[C1]), (C2, sol[C2]), (C3, sol[C3]), (C4, sol[C4])]

f_TE = sp.Matrix([-(V.subs(constantes)).subs(x, 0),   # Yi  se evaluan las reacciones verticales y los
                  +(M.subs(constantes)).subs(x, 0),   # Mi  momentos en los apoyos y se les multiplica
                  +(V.subs(constantes)).subs(x, L),   # Yj  por -1 para estimar la fuerza nodal 
                  -(M.subs(constantes)).subs(x, L)])  # Mj  equivalente

Se imprime el vector de fuerzas nodales equivalentes para la viga de Timoshenko-Ehrenfest:

In [7]:
imprimir(r'\boldsymbol{f}_{TE} = ', f_TE, grande=True)
#sp.pprint(simplify(f_TE))

<IPython.core.display.Math object>

Se imprime el vector de fuerzas nodales equivalentes para la viga de Euler-Bernoulli:

In [8]:
f_EB = f_TE.limit(beta,0)
imprimir(r'\boldsymbol{f}_{EB} = ', f_EB, grande=True)

<IPython.core.display.Math object>

## Se calcula la matriz de masa consistente $\boldsymbol{M}$

La matriz de masa se separa en dos clases de matrices:
\begin{equation*}
\boldsymbol{M} = \boldsymbol{M}_{\rho A} +  \boldsymbol{M}_{\rho I}
\end{equation*}

La matriz $\boldsymbol{M}_{\rho A}$ es la asociada a la inercia translacional

La matriz $\boldsymbol{M}_{\rho I}$ es la asociada a la inercia rotacional. Aquí $I$ es el momento de inercia alrededor del eje de rotación, que en este caso es la fibra neutra de la viga. 

Ver:
* https://en.wikipedia.org/wiki/Moment_of_inertia
* https://en.wikipedia.org/wiki/Rotational_energy

### Se calcula la matriz de masa asociada a la inercia translacional $\boldsymbol{M}_{\rho A}$

In [9]:
rho, A = sp.symbols('rho A')
N = sp.Matrix.vstack(N_u, N_w)
M_TE_rhoA = sp.integrate(rho*A*N.T*N * L/2, (xi, -1, 1))

tmp = rho*A*L/(210 * (1 + beta)**2)
imprimir(r'\boldsymbol{M}_{TE,\rho A} = \frac{\rho A L}{210 (1+\beta)^2}', M_TE_rhoA/tmp)

<IPython.core.display.Math object>

In [10]:
M_EB_rhoA = M_TE_rhoA.limit(beta, 0)
tmp = rho*A*L/420
imprimir(r'\boldsymbol{M}_{EB,\rho A} = \frac{\rho A L}{420}', M_EB_rhoA/tmp)

<IPython.core.display.Math object>

### Se calcula la matriz de masa asociada a la inercia rotacional $\boldsymbol{M}_{\rho I}$

In [11]:
rho, I = sp.symbols('rho I')
N = sp.Matrix.vstack(N_t)
M_TE_rhoI = sp.integrate(rho*I*N.T*N * L/2, (xi, -1, 1))

tmp = rho*I/(30*L*(1 + beta)**2)
imprimir(r'\boldsymbol{M}_{TE,\rho I} = \frac{\rho I}{30 (1+\beta)^2 L}', M_TE_rhoI/tmp)

<IPython.core.display.Math object>

Es importante anotar que la matriz de inercia rotacional $\boldsymbol{M}_{EB,\rho I}$ usualmente se desprecia en la teoría de Euler-Bernoulli:

In [12]:
M_EB_rhoI = M_TE_rhoI.limit(beta, 0)
tmp = rho*I/(30*L)
imprimir(r'\boldsymbol{M}_{EB,\rho I} = \frac{\rho I}{30 L}', M_EB_rhoI/tmp)

<IPython.core.display.Math object>